In [1]:
{-# LANGUAGE OverloadedStrings #-}

import qualified Data.ByteString as B
import Data.ByteString.UTF8 (fromString, toString)
import qualified Data.ByteString.Char8 as B (lines)
import qualified Data.Attoparsec.ByteString as A
import qualified Data.Attoparsec.ByteString.Char8 as AC

input <- B.lines <$> B.readFile "../input/day7.txt"

In [2]:
import Control.Applicative ((<|>))
import qualified Data.Set as S
import Data.List (foldl')

data Sequence a = Regular a | Hypernet a deriving (Eq, Show)

instance Functor Sequence where
    fmap f (Regular  x) = Regular  $ f x
    fmap f (Hypernet x) = Hypernet $ f x
    

bothmap :: (a -> b) -> (a -> b) -> Sequence a -> Sequence b
bothmap f g (Regular  x) = Regular $ f x
bothmap f g (Hypernet x) = Hypernet $ g x

parseHypernet = Hypernet <$> (AC.char '[' *> A.many1' AC.letter_ascii <* AC.char ']')
parseRegular  = Regular  <$> A.many1' AC.letter_ascii

parseIPv7 = A.many1' (parseRegular <|> parseHypernet)
    

fourChars s@[a,b,c,d] = [s]
fourChars s           = take 4 s : fourChars (tail s)

isABBA [a,b,c,d] = a==d && b==c && a/=b

threeChars s@[a,b,c] = [s]
threeChars s           = take 3 s : threeChars (tail s)

isABA [a,b,c] = ((a==c && a/=b), (a,b))
isBAB [a,b,c] = ((a==c && a/=b), (b,a))


result = map (map (fmap fourChars) . either error id . A.parseOnly parseIPv7) input


check = fmap (or . map isABBA)

hasABBA = map (map check) result

length $ filter (\r -> Regular True `elem` r && Hypernet True `notElem` r) hasABBA

separate = foldl' (\acc s -> case s of
    Regular  v -> (S.union (fst acc) v, snd acc)
    Hypernet v -> (fst acc, S.union (snd acc) v)) (S.empty,S.empty)



result' = map (map (fmap threeChars) . either error id . A.parseOnly parseIPv7) input
check' = bothmap (S.fromList . map snd . filter fst . map isABA) (S.fromList . map snd . filter fst . map isBAB)
checked' = filter (\(x,y) -> S.intersection x y /= S.empty) $ map (separate .map check') result'

length checked'

110

242